In [1]:
#Scraping
import requests
from bs4 import BeautifulSoup as bs

#notoverdoing it
import time

#working on scraped Data
import re

#dataset creation and export
import pandas as pd

In [ ]:
#function to delete possible double entries

def DeleteDoubles(L):
    seen = set()
    seen2 = set()
    seen_add = seen.add
    seen2_add = seen2.add
    for item in L:
        if item in seen:
            seen2_add(item)
        else:
            seen_add(item)
    return list(seen),list(seen2)

In [2]:
CountryList = [["Germany",'https://www.tripadvisor.com/Hotels-g187275-Germany-Hotels.html'],
               ["France",'https://www.tripadvisor.com/Hotels-g187070-France-Hotels.html'],
               ["Italy",'https://www.tripadvisor.com/Hotels-g187768-Italy-Hotels.html'],
               ["Netherlands",'https://www.tripadvisor.com/Hotels-g188553-The_Netherlands-Hotels.html'],
               ["Belgium",'https://www.tripadvisor.com/Hotels-g188634-Belgium-Hotels.html'],
               ["Luxembourg",'https://www.tripadvisor.com/Hotels-g190340-Luxembourg-Hotels.html'],
               ["Liechtenstein",'https://www.tripadvisor.com/Hotels-g190357-Liechtenstein-Hotels.html'],
               ["Portugal",'https://www.tripadvisor.com/Hotels-g189100-Portugal-Hotels.html'],
               ["Spain","https://www.tripadvisor.com/Hotels-g187427-Spain-Hotels.html"],
               ["Ireland",'https://www.tripadvisor.com/Hotels-g186591-Ireland-Hotels.html'],
               ["NorthernIreland","https://www.tripadvisor.com/Hotels-g186469-Northern_Ireland-Hotels.html"],
               ["Scotland","https://www.tripadvisor.com/Hotels-g186485-Scotland-Hotels.html"],
               ["Wales","https://www.tripadvisor.com/Hotels-g186425-Wales-Hotels.html"],
               ["England","https://www.tripadvisor.com/Hotels-g186217-England-Hotels.html"],
               ["Switzerland","https://www.tripadvisor.com/Hotels-g188045-Switzerland-Hotels.html"],
               ["Austria", "https://www.tripadvisor.com/Hotels-g190410-Austria-Hotels.html"],
               ["SanMarino","https://www.tripadvisor.com/Hotels-g187808-San_Marino-Hotels.html"],
               ["Sweden","https://www.tripadvisor.com/Hotels-g189806-Sweden-Hotels.html"],
               ["Norway","https://www.tripadvisor.com/Hotels-g190455-Norway-Hotels.html"],
               ["Finland","https://www.tripadvisor.com/Hotels-g189896-Finland-Hotels.html"],
               ["Denmark","https://www.tripadvisor.com/Hotels-g189512-Denmark-Hotels.html"],
               ['Monaco','https://www.tripadvisor.com/Hotels-g190405-Monaco-Hotels.html'],
               ['Estonia','https://www.tripadvisor.com/Hotels-g274952-Estonia-Hotels.html'],
               ["Latvia",'https://www.tripadvisor.com/Hotels-g274960-Latvia-Hotels.html'],
               ["Lithuania","https://www.tripadvisor.com/Hotels-g274947-Lithuania-Hotels.html"],
               ["Poland","https://www.tripadvisor.com/Hotels-g274723-Poland-Hotels.html"],
               ["CzechRepublik",'https://www.tripadvisor.com/Hotels-g274684-Czech_Republic-Hotels.html'],
               ["Slovakia","https://www.tripadvisor.com/Hotels-g274922-Slovakia-Hotels.html"],
               ["Hungary","https://www.tripadvisor.com/Hotels-g274881-Hungary-Hotels.html"],
               ["Croatia",'https://www.tripadvisor.com/Hotels-g294453-Croatia-Hotels.html'],
               ["Slovenia",'https://www.tripadvisor.com/Hotels-g274862-Slovenia-Hotels.html'],
               ["Greece",'https://www.tripadvisor.com/Hotels-g189398-Greece-Hotels.html'],
               ["Romania",'https://www.tripadvisor.com/Hotels-g294457-Romania-Hotels.html'],
               ["Bulgaria",'https://www.tripadvisor.com/Hotels-g294451-Bulgaria-Hotels.html'],
               ["Cyprus",'https://www.tripadvisor.com/Hotels-g190372-Cyprus-Hotels.html'],
               ["Malta",'https://www.tripadvisor.com/Hotels-g190311-Malta-Hotels.html'],
               ["Iceland","https://www.tripadvisor.com/Hotels-g189952-Iceland-Hotels.html"]]

In [ ]:
for line in CountryList:
    start= line[1]

    #initialise lists
    CityList =[]
    ll=[]

    #set first page as soup first
    ro= requests.get(start)
    so= bs(ro.content, 'lxml')

    #for first page, get names of cities
    for i in range(len(so.find_all('a', {'class':'linkText'}))):
        CityList.append(f"https://www.tripadvisor.com"+re.search('/Hotels(.*).html', str(so.find_all('a', {'class':'linkText'})[i]))[0])


    #split start link so page #s can be added 
    StrSpl=start.split("-",2)
    
    #get links for all other pages using information from first page and store in temporary list
    #to find number of pages find amount of page numbers displayed, index last one [-1], get content
    if len(so.find_all('div', {'class':'pageNumbers'}))>0:
        for i in range(int(so.find_all('div', {'class':'pageNumbers'})[0].findChildren('a')[-1].text)-1):
            ll.append(f'{StrSpl[0]}-{StrSpl[1]}-oa{(i+1)*20}-{StrSpl[2]}')

    #use list of other pages
        for e in ll:
    
        #set each page as soup respectively
            ro= requests.get(e)
            so= bs(ro.content, 'lxml')
    
        #and get links for all cities on page
            for i in range(len(so.find_all('a', {'class':'city'}))):
                CityList.append(f"https://www.tripadvisor.com"+re.search('/Hotels(.*).html', str(so.find_all('a', {'class':'city'})[i]))[0])


    #export to csv via pd dataframe
    df = pd.DataFrame(data={"link": CityList})
    df.to_csv(f"CityList{line[0]}.csv", sep=',',index=False)
    
    CityListfromCSV = pd.read_csv(f'/Users/FelixHoffmann/Desktop/MSc Thesis/CityList{line[0]}.csv')['link'].values.tolist()
    
    
    #get links to hotels from list of cities

    HotelLinks=[]
    BrokenCities=[]
    EmptyCities=[]
    MissedLinks=[]

    for g in CityListfromCSV:
    
        #create empty temp list
        l=[]
        print(g)
        #get soup of city overview
        try:
            r_ui = requests.get(g)
            s_ui = bs(r_ui.content, 'lxml')
        except:
            MissedLinks.append(g)
            time.sleep(5)
        
        #split initial link and add pagecount (dependent on total available hotels in region)
        StrSpl=g.split("-",2)

        #for #total // 30
        #If amount of properties is shown
        if len(s_ui.find_all('div', {'class':'_2AGh2u0d'}))>0:
            #and that amount of properties has numbers and not only text
            if len(re.findall(r'\d+', s_ui.find_all('div', {'class':'_2AGh2u0d'})[0].text.replace(",","")))>0:
            
                # range is the first number displayed in the text 'x of y properties is available in Z'
                for i in range(((int(re.findall(r'\d+', s_ui.find_all('div', {'class':'_2AGh2u0d'})[0].text.replace(",",""))[0])//30)+1)):
            
                #create temporary list l with links to pages for one region
                    l.append(f'{StrSpl[0]}-{StrSpl[1]}-oa{i*30}-{StrSpl[2]}')
    
                #for each page of 30 entries
                for entry in l:
                #look at page and create soup
                    try:
                        r= requests.get(entry)
                        s= bs(r.content, 'lxml')
                    except:
                        MissedLinks.append(entry)
                        time.sleep(5)
            
                    #for each hotel in the actual-search-results (not advertised section)
                    if len(s.find_all('div', {'id':'taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0'}))>0:
                            for t in range(len(s.find_all('div', {'id':'taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0'})[0].findChildren('div', {'class':'listing_title'}))):
                                HotelLinks.append(f"https://www.tripadvisor.com"+re.search('/Hotel_Review(.*).html', str(s.find_all('div', {'id':'taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0'})[0].findChildren('div', {'class':'listing_title'})[t]))[0])
        
            else:
                EmptyCities.append(g)
        else:
            BrokenCities.append(g)


    #delete doubles, only continue with cleaned list
    CleanedList, DoublesList= DeleteDoubles(HotelLinks)

    #export to csv via pd dataframe
    df = pd.DataFrame(data={"link": CleanedList})
    df.to_csv(f"HotelList{line[0]}.csv", sep=',',index=False)